In [7]:
#import package, define function 

# Alt + F to expand
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
#define similariry function 
import numpy as np
import pandas as pd
import math
import re
import time
import os
import networkx as nx
import pandas as pd
import datetime 
import pymssql
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '172.16.3.105', 'port': 9200}])

def dotproduct_(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def dotproduct(v1, v2):
    tmp = 0
    for a, b in zip(v1, v2):
        if a >= 0 and b >= 0:
            tmp += (a * b)
    return tmp   


def length(v):
    return math.sqrt(dotproduct(v, v))

def similar(v1, v2):
    return (dotproduct(v1, v2) / (length(v1) * length(v2)))

def length2(v1, v2):
    return math.sqrt(sum((a-b)**2 for a, b in zip(v1, v2)))
def length3(v1, v2):
    tmp = 0
    for a, b in zip(v1, v2):
        if a >= 0 and b >= 0:
            tmp += (a - b)**2
    return math.sqrt(tmp)       

def rmlistchar(text):
    a = re.findall(r"[^][)\"(']", text)
    a = ''.join(a)
    return a

#remove accents vietnamese
def remove_accents(text):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
#     s1.encode('utf-8')
#     s0.encode('utf-8')
    if text == np.nan or text == np.NAN or text == np.NaN:
        return text
    else:    
        s = ''
        text = str(text)
        text = text.decode('utf-8')

        for c in text:
            if c in s1:
                s += s0[s1.index(c)]
            else:
                s += c
        return s.strip()
    
# remove_accents('Hỗ trợ 4G hôm nãy đi chợi Hỗ trợ Hỗ trợ')   


def list_similar(item, numofpro, dataframe):
    list_similarity = dataframe[str(item)].sort(inplace = False)[: numofpro + 1]# sort by distance
    if str(item) in list_similarity:
        return list_similarity.drop([str(item)])
    #item_similarity = list_similarity.index.tolist()
    #item_similarity = map(lambda x: np.int(x),item_similarity)
    #del list_similarity[list_similarity.index(item)]
    else:
        return list_similarity#, item_similarity[1: numofpro+1]

def list_best_sale(category, numofpro, dataframe):
    #dataframe = dataframe.fillna(0)
#     best_sell = dataframe.loc[np.int(category)].sort_values(ascending = False, inplace = False)
#     item_best_sell = best_sell.index.tolist()
#     return item_best_sell[1: numofpro+1]# , best_sell
    return dataframe.loc[category].sort_values(ascending = False, inplace = False)[0: numofpro]

# Tracking...

In [8]:
#connect to database
'''
Created on 27 Mar 2016
@author: lbtanh
'''
import pymssql
from os import getenv
# -*- coding: utf-8 -*-
import tornado.ioloop
import tornado.web
import tornado.httpserver
import tornado.httpclient
import tornado.netutil
import sys
from tornado.ioloop import IOLoop, PeriodicCallback
import datetime
from threading import Thread
import time
# from tornado.options import define, options
# define("port", default=8897, help="run on the given port", type=int)


class Test(tornado.web.RequestHandler):
#     def __init__(self):
#         self.a = HandClass(self)
#     def cong(self, a, b):
#         return a+b
    
    def get(self):
        self.write("Thegioididong.com")

class TrackingViews(tornado.web.RequestHandler):

    def call(self):
        return cong(1, 2)
    def get(self):
#         self.write(str(test))
#         c.a.cong(3, 4)
        try:
            producID = int(self.get_argument('ProductID', None))
            num = int(self.get_argument('num', None))
            self.write(str(out_graph(info_product, producID, num)))
        except Exception as error:
            self.write("unable to get tracking list by" + producID +" Error: "  + repr(error))          

            
class Recommend(tornado.web.RequestHandler):
    def get(self):
        try:
            cookie = self.get_argument('Cookie', None)
            numofpro = int(self.get_argument('num', None))
            numofdays= int(self.get_argument('days', None))
            tmp1, tmp2 =list_by_statsmodels(cookie, tracking_DF, numofdays, info_product, numofpro)
            self.write('{recommend:'+str(tmp1))
            self.write(',mostViews:'+str(tmp2)+ '}')
        except Exception as error:
            self.write("unable to get recommendation list by " + cookie +" Error: "  + repr(error))                 
            
            
def preprocessing_df(dataframe):
    tracking_data = dataframe
    # preprocessing tracking dataframe
    tracking_data = tracking_data.dropna(subset=['ProductID'])
#     tracking_data = tracking_data.sort(columns=['COOKIE', 'CREATEDTIME'])
    list_product_ID_tracking_data = map(lambda x: int(x), set(tracking_data.ProductID))
    df = pd.DataFrame(list_product_ID_tracking_data, columns=['ProductID'])
    df['CateID'] = df.ProductID.apply(get_cate_ID_by_proID, args=(0, es, ))
#     df['CateID'] = df['CateID'].apply(lambda x: int(x))
    df = df.dropna(how = 'any', subset=['CateID'])
    df = df[~df.CateID.isin(['3103', '2102'])]
    # df['NameCate']= df.CateID.apply(get_name_cate_by_cate_ID)
    # df['NameCate_']=df['NameCate'].apply(remove_accents)
    tracking_DF = tracking_data.merge(df, how='left', on = 'ProductID')
    return tracking_DF            
# ================================================================================
def cong(a, b):
    return a+b            
            
def connectDB():
    server = "10.1.12.197"
    user = "rd"
    password = "rd123456"
    conn = pymssql.connect(server, user, password, "RD")
    cursor = conn.cursor(as_dict = True)
    return cursor
# select tracking data

def get_DB_tracking_data(num_of_days):
    try:
        cursor = connectDB()
        # tracking_data = 'use RD SELECT [COOKIE] ,[SESSION],[CREATEDTIME],[URL]  FROM [RD].[dbo].[TRACKINGDATA] where SITEID = 8'
        tracking_data = 'use RD SELECT [ID], [COOKIE] ,[SESSION],[CREATEDTIME], \
        [RD].[dbo].[TRACKINGDATA].[URL], vuivui.Product.ProductID \
        FROM [RD].[dbo].[TRACKINGDATA] \
        left join [RD].[vuivui].[Product] on vuivui.Product.RealUrl = RD.dbo.TRACKINGDATA.URL \
        where SITEID = 8 and DATEDIFF(day,CREATEDTIME, GETDATE()) <= %d' %(num_of_days)
        cursor.execute(tracking_data)
        tracking_data = cursor.fetchall()
        tracking_data = pd.DataFrame(tracking_data)
        cursor.close()
        current_ID = int(tracking_data.tail(1).ID.values)
    except:
        pass
    return tracking_data, current_ID

    
def get_DB_current(current_ID):
    cursor = connectDB()
    current_data = 'use RD SELECT [ID], [COOKIE] ,[SESSION],[CREATEDTIME], \
    [RD].[dbo].[TRACKINGDATA].[URL], vuivui.Product.ProductID \
    FROM [RD].[dbo].[TRACKINGDATA] \
    left join [RD].[vuivui].[Product] on vuivui.Product.RealUrl = RD.dbo.TRACKINGDATA.URL \
    where SITEID = 8 and ID > %d' %(current_ID)
    cursor.execute(current_data)
    current_data = cursor.fetchall()
    current_data = pd.DataFrame(current_data)
    cursor.close()
    if current_data.shape[0] == 0:
        return current_data, current_ID
    current_ID = int(current_data.tail(1).ID.values)
    return current_data, current_ID

# ================================================================================


def get_cate_ID_by_proID(productID, position, es):
    try:
        #connect to our cluster
        from elasticsearch import Elasticsearch
        es = Elasticsearch([{'host': '172.16.3.105', 'port': 9200}])
        # res = requests.get('http://172.16.3.105:9200/index_product/product/86734')
        # print(res.content)
        pathCate = es.get(index="index_product", doc_type="product", id=str(int(productID)))['_source']['categoryPath']
        listCate = pathCate.split(" ")
        return listCate[position]
    except Exception as error:
        return np.nan



def get_name_cate_by_cate_ID(cate_ID):
    try: 
        url = 'http://webservice.vuivui.com/productsvc.asmx/GetProductCategoryByCate?intSiteID=8&intCategoryID='
        url = url + str(int(cate_ID))
        res = requests.get(url)
        xml = ET.fromstring(res.content)
        return xml[0][1].text
    except Exception as error:
        return ""
# ================================================================================

def get_DB_test():
    cursor = connectDB()
    hot_product = "SELECT TOP 1 *  FROM [RD].[dbo].[TRACKINGDATA] where CREATEDTIME >= '2017-05-11'  ORDER BY id DESC"    
    cursor.execute(hot_product)
    hot_product = cursor.fetchall()
    hot_product = pd.DataFrame(hot_product)
    cursor.close()
    return hot_product


def get_DB_hot_product():
    cursor = connectDB()
    hot_product = 'use RD SELECT RD.dbo.TRACKINGDATASUMMARIZE.ProductID, Price, SUM(Value) as Views, vuivui.Product.CategoryID FROM RD.dbo.TRACKINGDATASUMMARIZE join vuivui.Product on vuivui.Product.ProductID = RD.dbo.TRACKINGDATASUMMARIZE.ProductID where Type = 1  group by RD.dbo.TRACKINGDATASUMMARIZE.ProductID, RefProductID, Price, ProductName, vuivui.Product.CategoryID order by vuivui.Product.CategoryID,SUM(Value) desc'
    cursor.execute(hot_product)
    hot_product = cursor.fetchall()
    cursor.close()
    return pd.DataFrame(hot_product)


# select tracking table
def get_DB_views(info_product):
    cursor = connectDB()
    sql = "use RD DECLARE @day INT=  CAST(convert(varchar(8), GETDATE(), 112) AS VARCHAR(8)) \
    +CAST(DATEPART(HOUR, GETDATE()) AS VARCHAR(2)); \
    SELECT RD.dbo.TRACKINGDATASUMMARIZE.ProductID, RefProductID, Price, \
    ProductName, SUM(Value) as Views, vuivui.Product.CategoryID \
    FROM RD.dbo.TRACKINGDATASUMMARIZE \
    join vuivui.Product on vuivui.Product.ProductID = RD.dbo.TRACKINGDATASUMMARIZE.RefProductID \
    where Type = 4 and @day - DayKey < 20000 \
    group by RD.dbo.TRACKINGDATASUMMARIZE.ProductID, RefProductID, \
    Price, ProductName, vuivui.Product.CategoryID \
    order by SUM(Value) desc"
    
    cursor.execute(sql)
    a = cursor.fetchall()
    #preprocessing
    testA = pd.DataFrame(a)
    testA = testA[testA['Price'] != -1]
    testA = testA[testA['Price'] != 0]
    #preprocessing
    testA = pd.merge(testA, info_product.iloc[:,],how = 'left', left_on = 'ProductID',right_on = 'ProductID')
    testA.rename(columns={'Price_x':'RefPrice', 'Price_y':'ProductPrice', 'CategoryID_x':'RefCategoryID', 'CategoryID_y':'CategoryID', 'ProductName_x':'RefProductName', 'ProductName_y':'ProductName'}, inplace=True)

    # drop NaN value in columns RefProductID
    testA = testA.dropna(axis = 0, how='any', subset = ['RefProductID', 'ProductID'])
    testA = testA.loc[:,['ProductID', 'ProductName', 'ProductPrice', 'RefProductID', 'RefProductName','RefPrice', 'Views', 'CategoryID', 'RefCategoryID']]

    testA[['ProductName', 'RefProductName']] = testA[['ProductName', 'RefProductName']].applymap(remove_accents)
    testA[['ProductName', 'RefProductName']] = testA[['ProductName', 'RefProductName']].applymap(rmlistchar)
        #     test = demo[['ProductID','RefProductID', 'Views']]
    return testA

#get info product
def get_DB_all_product():
    try:
        cursor = connectDB()
        produc_cate = 'use RD SELECT [ProductID] ,[ProductName] ,[Price] \
        FROM [RD].[vuivui].[Product]'
        cursor.execute(produc_cate)
        info_product = cursor.fetchall()
        info_product = pd.DataFrame(info_product)
        info_product  = info_product.drop_duplicates()
        cursor.close()
        info_product['CategoryID'] = info_product.ProductID.apply(get_cate_ID_by_proID, args=(-1, es, ))
        info_product = info_product[info_product['CategoryID'] > 1]
        info_product['CategoryID'] = info_product.CategoryID.apply(lambda x: int(x))
    except:
        pass
    return info_product
# input: dataframe testA
# create graph by ID
def create_graph(test):
    g = nx.DiGraph((raw['ProductID'], raw['RefProductID'], {'weight': raw['Views']}) for index, raw in test.iterrows())
    return g

# create graph by name
def create_graph1(test):
    g = nx.DiGraph((raw['ProductName'], raw['RefProductName'], {'weight': raw['Views'], 'label':str(raw['Views'])}) for index, raw in test.iterrows())
    return g

#     def inital_graph(self, categoryID, testA):
#         for cate in categoryID:
#             exec('cate_'+str(cate)+ '= testA[(testA.CategoryID =='+str(cate)+ ') & (testA.RefCategoryID ==' +str(cate)+')]')
#             exec('global g_'+str(cate))
#             exec('g_'+str(cate)+ '= create_graph(cate_'+str(cate)+')')
#         return    

# recommended by tracking data
def out_graph(info_product, productid, numofpro, typegraph = 'out'):
    # return list of tuples which was sorted by weight out edges
    try:
        cate_pro = int(info_product['CategoryID'][info_product['ProductID']==productid])
    #     df = df[df['CategoryID']==cate_pro]
    #     g = create_graph(df[:-1])
        exec('g=g_'+str(cate_pro))
        if typegraph == 'out':
            if productid in pd.DataFrame(g.out_edges()).iloc[:][0].tolist():
                df_weight = pd.DataFrame(g.out_edges(productid, data='weight'))
                df_weight.columns = ['ProductID','RefProductID', 'Views']
                df_weight=df_weight.sort(columns = ['Views'], ascending = False)[:numofpro]
            else:
                return []
    #         tmp = testA[testA.columns][testA.RefProductID.isin(df_weight['RefProductID'])]
    #         return tmp[tmp.ProductID == productid]
        else:
            if productid in pd.DataFrame(g.out_edges()).iloc[:][0].tolist():
                df_weight = pd.DataFrame(g.in_edges(productid, data='weight'))
                df_weight.columns = ['ProductID','RefProductID', 'Views']
                df_weight=df_weight.sort(columns = ['Views'], ascending = False)[:numofpro]
            else:
                return []
    #         tmp = testA[testA.columns][testA.ProductID.isin(df_weight['ProductID'])]
    #         return tmp[tmp.RefProductID == productid]
        #sorted by 
        return df_weight[['RefProductID', 'Views']].to_json(orient='values') 
    except:
        pass

# it can work with lambda function
'''
cookie: cookie of user
tracking_DF: dataframe tracking data
ndays: number of day to processing
'''
def list_by_statsmodels(cookie, tracking_DF, ndays, info_product, numofpro):
    tmp = tracking_DF[tracking_DF['COOKIE']==cookie]
#     f = {'SESSION':['nunique'], 'ProductID':['nunique', 'count'], 'CateID': ['nunique']}
    f = {'SESSION':['nunique'],'ProductID':['count']}
    stat_tracking = tmp.groupby(['COOKIE','ProductID']).agg(f)
    stat_tracking = pd.DataFrame(stat_tracking)
    stat_tracking = stat_tracking.reset_index()
    stat_tracking.columns = ['COOKIE', 'ProductID', 'Count_Session', 'Views']
    list_relative = list(stat_tracking[stat_tracking.Count_Session == 2].ProductID.values)
    list_relative.extend(list(stat_tracking[stat_tracking.Views >= 3].ProductID.values))
    list_recommend = list(stat_tracking[stat_tracking.Count_Session > 2].ProductID.values)
    list_recommend.extend(list(stat_tracking[stat_tracking.Views > 3].ProductID.values))
    list_recommend = map(lambda x: int(x), list_recommend)
    
    df_tmp = tracking_DF
    df_tmp['DAYS'] = datetime.date.today() - df_tmp.CREATEDTIME
    tracking_DF_ndays = df_tmp[df_tmp['DAYS'] <= datetime.timedelta(ndays)]
    list_tmp = unique_list_order(tracking_DF_ndays[tracking_DF_ndays['COOKIE']==cookie].ProductID.values)
    list_relative.extend(list_tmp)
    list_relative = map(lambda x: int(x), list_relative)
    list_relative=out_graph_by_list(info_product, list(set(list_relative)), numofpro)
    list_relative = map(lambda x: int(x), list_relative)
    return  list_relative, list(set(list_recommend))    
    
    
def unique_list_order(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def unique_list_order_(list_):
    list_ = np.array(list_)
    _, idx = np.unique(list_, return_index=True)
    return list_[np.sort(idx)]
# return list of recommended product by list viewed product
def out_graph_by_list(info_product, list_productid, numofpro):
    try:
        list_results = []
        for productid in list_productid:
            cate_pro = int(info_product['CategoryID'][info_product['ProductID']==productid])
            exec('g=g_'+str(cate_pro))
            if productid in pd.DataFrame(g.out_edges()).iloc[:][0].tolist():
                df_weight = pd.DataFrame(g.out_edges(productid, data='weight'))
                df_weight.columns = ['ProductID','RefProductID', 'Views']
                df_weight=df_weight.sort(columns = ['Views'], ascending = False)[:numofpro]
                list_results.extend(df_weight.RefProductID.values)
                list_results = unique_list_order(list_results)
    except:
        pass
    return list_results



def inital_():
    try:
    #     print datetime.datetime.now()
        global info_product
        global testA
        global categoryID
        info_product = get_DB_all_product()
        testA = get_DB_views(info_product)
        categoryID = info_product.CategoryID.unique().tolist()
        #         self.g_all = create_graph(testA)
        for cate in categoryID:
            exec('cate_'+str(cate)+ '= testA[(testA.CategoryID =='+str(cate)+ ') & (testA.RefCategoryID ==' +str(cate)+')]', globals())
            exec('g_'+str(cate)+ '= create_graph(cate_'+str(cate)+')', globals())
    except:
        pass


# ================================================================================
class Application(tornado.web.Application):
    def __init__(self):
        handlers = [
                  (r"/trackingview", TrackingViews),
                (r"/recommend", Recommend),
                  (r"/", Test),              
        ]
        settings = {}
        tornado.web.Application.__init__(self, handlers, **settings)


def thread_get_resys_by_views(threadname):
    while 1:
        inital_()
        time.sleep(86400)   

def thread1(threadname):
    #global a       # Optional if you treat a as read-only
    while 1:
        http_server = tornado.httpserver.HTTPServer(Application())
        http_server.listen(8808)

def inital_test():
    global test
    test = get_DB_test()
#     time.sleep(2)
def thread3(threadname):
#     global test
    while 1:
        inital_test()
        time.sleep(3)  

def tracking_get_DB():
    global tracking_DF, current_ID
    tracking_data, current_ID = get_DB_tracking_data(14)
    tracking_DF = preprocessing_df(tracking_data)
    
def thread_tracking_get_DB(threadname):
    while 1:
        tracking_get_DB()
        time.sleep(1000)

def tracking_update_DB():
    global current_DB, current_ID, tracking_DF
    current_DB, current_ID = get_DB_current(current_ID)
    if current_DB.shape[0]!=0:
#         tracking_data = tracking_data.append(current_DB)
        current_DB = preprocessing_df(current_DB)
        tracking_DF=tracking_DF.append(current_DB)

def thread_tracking_update_DB(threadname):
    while 1:
        tracking_update_DB()
        time.sleep(2)  
        
if __name__ == "__main__":
    
#     inital_()
#     http_server = tornado.httpserver.HTTPServer(Application())
#     http_server.listen(8808)
#     PeriodicCallback(inital_, 4*60*1000).start()
    info_product = 0
    testA = 0
    categoryID = 0
    test = 0
    current_DB = 0
    tracking_DF = 0
    current_ID = 0
#     tracking_data = 0
    
    thread1 = Thread( target=thread1, args=("Thread-1", ) )
    thread_get_resys_by_views = Thread( target=thread_get_resys_by_views, args=("Thread get item views", ) )
    thread3 = Thread( target=thread3, args=("Thread-3", ) )
    thread_tracking_get_DB = Thread( target=thread_tracking_get_DB, args=("Thread get all tracking data DB", ))
    thread_tracking_update_DB = Thread( target=thread_tracking_update_DB, args=("Thread update tracking data DB", ))


    start = time.time()
    thread1.start()
    time.sleep(1)
    thread_get_resys_by_views.start()
    time.sleep(2)
    time1 = time.time() - start
    
    start = time.time()
    tracking_get_DB()
#     thread_tracking_get_DB.start()
#     time.sleep(250)
    time.sleep(1)
    thread_tracking_update_DB.start()
    
    time2 = time.time() - start

    

In [388]:
# tornado.ioloop.IOLoop.instance().stop()

In [258]:
info_product.to_csv('info_product.csv')

In [43]:
tracking_DF.to_csv('tracking_DF.csv')

In [6]:

def read_DB_from_file():
    global tracking_DF, current_ID
    tracking_DF = pd.read_csv('tracking_DF.csv')
    current_ID = int(tracking_DF.tail(1).ID.values)

In [7]:
read_DB_from_file()

In [3]:
current_ID

268123358

In [420]:
# a9d84f26-0e54-4c24-b40e-1f6713987677
# 2b05443b-ab18-4437-9090-c434aba3d7da
def get_basket_customer():
    cursor = connectDB()
    basket_customer = "use RD SELECT ID, SESSION, PRODUCTID, NUMBERITEM FROM [RD].[dbo].[TRACKINGDATA] where SITEID = 8 and PRODUCTID != -1 and [COOKIE] = 'a9d84f26-0e54-4c24-b40e-1f6713987677' order by ID desc"    
    cursor.execute(basket_customer)
    basket_customer = cursor.fetchall()
    basket_customer = pd.DataFrame(basket_customer)
    cursor.close()
    return basket_customer

def get_record():
    # filter by session
    # list_order_id = []
    transactionList = list()
    itemSet = set()
    basket_customer =  get_basket_customer()
    for session in list(basket_customer.SESSION.unique()):
        basket_customer_session = basket_customer[basket_customer['SESSION']==session]
        data = basket_customer_session.drop_duplicates(subset = ['PRODUCTID'])
        for item in list(data[data['NUMBERITEM']< 0].PRODUCTID):
            data.loc[data.PRODUCTID == item, 'NUMBERITEM'] = basket_customer_session[basket_customer_session['PRODUCTID']==item].NUMBERITEM.sum()
        record = data[data['NUMBERITEM']>0]
        if record.shape[0] > 0:
            record1 = frozenset(record.PRODUCTID)
            yield record1
            
#             record = list(record.PRODUCTID.unique())
#             list_order_id.append(record)  
    # write to file record       
    # with open("list_to_file1.txt", "w") as output:
    #     for item in list_order_id:
    #         output.write('%s\n' %','.join(map(str,item)))    


start = time.time()
aaa = get_record()
time.time() - start    

7.295608520507812e-05

In [421]:
sum(1 for i in aaa)

28

In [409]:
a = (x*x for x in range(10))

In [411]:
sum(a)

285

In [404]:
tmpa.next()

frozenset({'107686',
           '76413',
           '76488',
           '77230',
           '77718',
           '79562',
           '79571',
           '79710',
           '81658',
           '83935',
           '90968',
           '91905'})

In [315]:
def getItemSetTransactionList(data_iterator):
        transactionList = list()
        itemSet = set()
        for record in data_iterator:
            transaction = frozenset(record)
            transactionList.append(transaction)
            for item in transaction:
                itemSet.add(frozenset([item]))              # Generate 1-itemSets
        return itemSet, transactionList

In [316]:
itemSet, transactionList = getItemSetTransactionList(tmpa)

In [318]:
itemSet


{frozenset({'82640'}),
 frozenset({'105771'}),
 frozenset({'96034'}),
 frozenset({'106249'}),
 frozenset({'83244'}),
 frozenset({'105139'}),
 frozenset({'76514'}),
 frozenset({'96096'}),
 frozenset({'81422'}),
 frozenset({'77146'}),
 frozenset({'84857'}),
 frozenset({'104896'}),
 frozenset({'79710'}),
 frozenset({'96416'}),
 frozenset({'108602'}),
 frozenset({'79490'}),
 frozenset({'81752'}),
 frozenset({'76446'}),
 frozenset({'85679'}),
 frozenset({'77154'}),
 frozenset({'85820'}),
 frozenset({'90520'}),
 frozenset({'86003'}),
 frozenset({'79832'}),
 frozenset({'79324'}),
 frozenset({'93229'}),
 frozenset({'79571'}),
 frozenset({'80570'}),
 frozenset({'103397'}),
 frozenset({'83949'}),
 frozenset({'84813'}),
 frozenset({'85984'}),
 frozenset({'79823'}),
 frozenset({'89090'}),
 frozenset({'76929'}),
 frozenset({'82178'}),
 frozenset({'82208'}),
 frozenset({'88360'}),
 frozenset({'91905'}),
 frozenset({'96411'}),
 frozenset({'102587'}),
 frozenset({'76356'}),
 frozenset({'76413'}),
 fro

In [307]:
itemSet.pop()

frozenset({'105139'})

In [90]:
# start = time.time()
# tracking_DF[tracking_DF['COOKIE']=='2b05443b-ab18-4437-9090-c434aba3d7da']
# time.time() - start

In [91]:
out_graph(info_product, 91060, 10)

In [92]:

list_productid = [84801, 90772, 88268]
out_graph_by_list(info_product, list_productid, 2)


[]

# Get history by cookie from DATATRACKING

# Get tree category

In [91]:
# get xml file
import requests
import xml.etree.ElementTree as ET
url2 = 'http://webservice.vuivui.com/productsvc.asmx/GetProductCategoryByCate?intSiteID=8&intCategoryID=166'
url = 'http://webservice.thegioididong.com/cmsservice/ProductSvc.asmx/GetAllProductCategory?intIsActived=1&intSiteID=8&strLanguageID=vi-VN'
res = requests.get(url, timeout=10)
xml = ET.fromstring(res.content)



# get list category path of product 
list_path = []
for child in xml.findall('{http://tempuri.org/}ProductCategoryBO'):
    tmp = child.find('{http://tempuri.org/}NodeTree').text
    list_path.append(tmp)
    
# delete ',' and convert to int
results = [[int(x) for x in path[1:-1].split(',')] for path in list_path if ',' in path[1:-1]]    


# to create graph
'''
G = nx.DiGraph()
for node in results:  
    a = [(node[i], node[i + 1]) for i in xrange(len(node) - 1)]
    for i in xrange(len(node) - 1):
        G.add_edge(node[i],node[i + 1])
'''  
G = nx.DiGraph()
for node in results: 
    G.add_path(node)
# nx.draw(G,with_labels = True)
# from matplotlib import pylab as plt
# plt.show()  


nx.nx_pydot.write_dot(G, 'graph_path_category.txt')
# run in window
# !os.system("dot -Tpng -ograph_path_category.png graph_path_category.txt")


df_path_category = pd.DataFrame(G.edges())
df_path_category.columns = ['source_ID', 'destination_ID']
df_path_category['source_name'] = df_path_category.source_ID.apply(get_name_cate_by_cate_ID)
df_path_category['destination_name'] = df_path_category.destination_ID.apply(get_name_cate_by_cate_ID)
# df_path_category.to_csv('df_path_category.csv')


# create graph by dataframe
G1 = nx.DiGraph((raw['source_name']+'\n'+str(raw['source_ID']), raw['destination_name']+'\n'+str(raw['destination_ID'])) for index, raw in df_path_category.iterrows())
# export text file to draw the graph by command line
nx.nx_pydot.write_dot(G1, 'graph_path_category_name.txt')

# thay doi thu tu cate theo cookie

In [1]:
import redis
from rediscluster import StrictRedisCluster
# Requires at least one node for cluster discovery. Multiple nodes is recommended.
startup_nodes = [{"host": "10.1.12.30", "port": "16379"}]
rc = StrictRedisCluster(startup_nodes=startup_nodes, decode_responses=True)

In [2]:
rc.hset('hash_test', 'key1', 'value1')

1L

In [4]:
rc.s

<bound method StrictRedisCluster.client_list of StrictRedisCluster<10.1.12.30:16379>>